In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import gensim.models
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('train/train_set.csv', sep='\t')
test = pd.read_csv('test/test.csv', sep='\t')

In [5]:
texts=pd.concat([train['text'],test['text']])
texts

0        2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1        4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2        7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3        7159 948 4866 2109 5520 2490 211 3956 5520 549...
4        3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
                               ...                        
49995    3725 4498 2282 1647 6293 4245 4498 3615 1141 2...
49996    4811 465 3800 1394 3038 2376 2327 5165 3070 57...
49997    5338 1952 3117 4109 299 6656 6654 3792 6831 21...
49998    893 3469 5775 584 2490 4223 6569 6663 2124 168...
49999    2400 4409 4412 2210 5122 4464 7186 2465 1327 9...
Name: text, Length: 250000, dtype: object

In [6]:
word2vec_model = gensim.models.Word2Vec(texts.values, size=63, iter=10,window=12, min_count=1,
                                        workers=8,seed=2020,sample=1e-5,sg=1,hs=1)
word2vec_model.save('texts.model')
print("success!!!!!!!!!")

success!!!!!!!!!


In [8]:
train['text']
def Get_vector(trace):    
    word2vec_model = gensim.models.Word2Vec.load('texts.model')
    index2word_set = set(word2vec_model.wv.index2word)
    startTime = time.time()
    vector_list = []
    num=0
    for cutWords in trace:
        i=0
        article_vector = np.zeros((word2vec_model.trainables.layer1_size))
        for cutword in cutWords:
            if cutword in index2word_set:
                article_vector = np.add(article_vector, word2vec_model.wv[cutword])
                i += 1
        cutWord_vector = np.divide(article_vector, i)
        vector_list.append(list(cutWord_vector))
        del cutWord_vector
        del article_vector
        num+=1
        if num %50000==0:
            print("前",num,"个耗时：",(time.time() - startTime))
    return pd.DataFrame(vector_list)
train_vector=Get_vector(train['text'].values)
test_vector=Get_vector(test['text'].values)


前 50000 个耗时： 1665.9355807304382
前 100000 个耗时： 3205.9941194057465
前 150000 个耗时： 4838.847198486328
前 200000 个耗时： 6518.758146047592
前 50000 个耗时： 1672.5965094566345


In [9]:
train_vector.to_csv("train_vec.csv",index=False)
test_vector.to_csv("test_vector.csv",index=False)

In [13]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(texts)
train_word_features = word_vectorizer.transform(train['text'].values)
test_word_features = word_vectorizer.transform(test['text'].values)




<200000x6977 sparse matrix of type '<class 'numpy.float64'>'
	with 56074040 stored elements in Compressed Sparse Row format>

In [16]:
x_train,x_test,y_train,y_test=train_test_split(train_word_features,train['label'],test_size=0.2,random_state=1)
clf = LogisticRegression(C=4)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
train_scores = clf.score(x_train, y_train)
print(train_scores, f1_score(y_pred, y_test, average='macro'))


D:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9484625 0.9218392965754442


In [17]:
label=clf.predict(test_word_features)
label=pd.DataFrame(label,columns=['label'])
label.to_csv("label.csv",index=False)

In [18]:
import lightgbm as lgb
from sklearn.metrics import f1_score
x_train,x_test,y_train,y_test=train_test_split(train_word_features,train['label'],test_size=0.2,random_state=1)

clf=lgb.LGBMClassifier(
    n_estimators=700,
    boosting_type='gbdt',
    learning_rate=0.1,
    objective='multiclass',
    num_class=14)

clf.fit(x_train,y_train,
         eval_set=[(x_train,y_train),(x_test,y_test)],
         verbose=100,
         early_stopping_rounds=100)

y_pred=clf.predict(x_test)
label=clf.predict(test_vector)
score=f1_score(y_test, y_pred, average='macro')
print("score:",score)

label=pd.DataFrame(label,columns=['label'])
label.to_csv("lgb_label.csv",index=False)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.754206	valid_1's multi_logloss: 1.09841
Early stopping, best iteration is:
[57]	training's multi_logloss: 0.241205	valid_1's multi_logloss: 0.320826


ValueError: Number of features of the model must match the input. Model n_features_ is 6977 and input n_features is 63 